# Set up notebook environment
## NOTE: Use a QIIME2 kernel

In [44]:
import os
import biom
import warnings
import pickle
import numpy as np
import pandas as pd
import qiime2 as q2
from biom import Table
from skbio import OrdinationResults
from skbio.stats import subsample_counts
from skbio.stats.distance import permanova, anosim, mantel
from skbio.stats.distance import DistanceMatrix
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.feature_table.actions import rarefy
from qiime2.plugins.diversity.actions import beta_group_significance
from qiime2.plugins.emperor.actions import biplot, plot
from qiime2.plugins.diversity.actions import (beta,
                                              beta_phylogenetic,
                                              pcoa)
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

from assets.step_wise_anova import run_stepwise_anova
from qiime2.plugins.fragment_insertion.actions import filter_features
warnings.filterwarnings("ignore", category=DeprecationWarning)

# helper functions
from assets.util import (mantel_matched, simulate_depth,
                        all_dists, nested_permanova)

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
%matplotlib inline


# Subset metadata to make paired files between extraction kits

In [45]:
SEQ_TYPE='shotgun'

In [46]:
# Read in sample metadata
md = pd.read_csv('HBM_Matrix_LCMS_1500/metadata_ms_HBM.tsv'.format(SEQ_TYPE=SEQ_TYPE), sep = '\t')
md = md[(md['ATTRIBUTE_type'] == 'saliva')]
md.to_csv('HBM_Matrix_LCMS_1500/metadata_saliva.tsv', sep='\t', index=False)
md

,sample_name,filename,ATTRIBUTE_storagebuffer,ATTRIBUTE_extraction_protocol,ATTRIBUTE_type,ATTRIBUTE_host_subject_id,ATTRIBUTE_sample_type2
0,361162770,361162770_GH7_01_64837.mzXML,etoh,Matrix,saliva,D,human_saliva_before_brushing
1,361162771,361162771_GE6_01_64793.mzXML,etoh,Matrix,saliva,C,human_saliva_after_brushing
2,361162790,361162790_GA8_01_64743.mzXML,etoh,Matrix,saliva,D,human_saliva_before_brushing
3,361162793,361162793_GE8_01_64795.mzXML,isopropanol,Matrix,saliva,D,human_saliva_before_brushing
4,361162798,361162798_GD9_01_64783.mzXML,isopropanol,Matrix,saliva,D,human_saliva_after_brushing
5,361162807,361162807_GC8_01_64769.mzXML,isopropanol,Matrix,saliva,D,human_saliva_before_brushing
6,361162809,361162809_GD8_01_64782.mzXML,isopropanol,Matrix,saliva,D,human_saliva_before_brushing
7,361164071,361164071_GA3_01_64738.mzXML,etoh,Matrix,saliva,A,human_saliva_after_brushing
8,361164072,361164072_GD3_01_64777.mzXML,isopropanol,Matrix,saliva,A,human_saliva_after_brushing
9,361164081,361164081_GA2_01_64737.mzXML,etoh,Matrix,saliva,A,human_saliva_before_brushing


# Stepwise RDA

## Shotgun data

In [47]:
# Import data
md_round1and2_bothPS_q2 = q2.Metadata.load('HBM_Matrix_LCMS_1500/metadata_saliva.tsv')

In [48]:
md_round1and2_bothPS_q2

Metadata
--------
36 IDs x 6 columns
filename:                      ColumnProperties(type='categorical')
ATTRIBUTE_storagebuffer:       ColumnProperties(type='categorical')
ATTRIBUTE_extraction_protocol: ColumnProperties(type='categorical')
ATTRIBUTE_type:                ColumnProperties(type='categorical')
ATTRIBUTE_host_subject_id:     ColumnProperties(type='categorical')
ATTRIBUTE_sample_type2:        ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation.

In [49]:
# Filter table
# table_shotgun_biom = table_shotgun.view(Table)
md_round1and2_bothPS_df = md_round1and2_bothPS_q2.to_dataframe()
# shared_ = list(set(table_shotgun_biom.ids()) & set(md_round1and2_bothPS_df.index))
md_round1and2_bothPS_df_shotgun = md_round1and2_bothPS_df#.reindex(shared_)
# table_shotgun_biom_bothPS = table_shotgun_biom.filter(shared_)
# keep_ = table_shotgun_biom_bothPS.ids('observation')[table_shotgun_biom_bothPS.sum('observation') > 0]
# table_shotgun_biom_bothPS.filter(keep_, axis='observation')

# # Import filtered table and re-indexed metadata file
# table_shotgun_bothPS = q2.Artifact.import_data('FeatureTable[Frequency]', table_shotgun_biom_bothPS)
md_round1and2_bothPS_df_shotgun_q2 = q2.Metadata(md_round1and2_bothPS_df_shotgun)

# # Generate distance matrices using 'all_dissts' utils
# rare_depth_shotgun = 1000
# dists_res_shotgun = all_dists(table_shotgun_bothPS,
#                       rare_depth_shotgun, tree_shotgun)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_shotgun = {}
# pcoa_res_shotgun['Jaccard'] = pcoa(dists_res_shotgun['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['Unweighted UniFrac'] = pcoa(dists_res_shotgun['Unweighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['Weighted UniFrac'] = pcoa(dists_res_shotgun['Weighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
# pcoa_res_shotgun['RPCA'] = dists_res_shotgun['RPCA'].biplot.view(OrdinationResults).samples
distance_matrix_Jaccard = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_jaccard_qiime2_noise_detect2_noblanks_HBM_R.qza')
pcoa_res_shotgun['Jaccard'] = pcoa(distance_matrix_Jaccard).pcoa.view(OrdinationResults).samples
distance_matrix_RPCA = q2.Artifact.load('HBM_Matrix_LCMS_1500/distance_matrix1500_canberra_qiime2_noise_detect2_noblanks_HBM_R.qza')
pcoa_res_shotgun['RPCA'] = pcoa(distance_matrix_RPCA).pcoa.view(OrdinationResults).samples
#names = {n: '14332.' + n for n in pcoa_res_shotgun['RPCA'].index} #appending "14332."to sample names in RPCA matrix
#pcoa_res_shotgun['RPCA'].rename(index=names, inplace=True)
#pcoa_res_shotgun["RPCA"] = pcoa_res_shotgun['RPCA'].loc[pcoa_res_shotgun['RPCA'].index.isin(pcoa_res_shotgun['Jaccard'].index)] #grabbing samples from the jaccard matrix only to use in RPCA #grabbing samples from the jaccard matrix only to use in RPCA

In [50]:
distance_matrix_RPCA

<artifact: DistanceMatrix uuid: c009c2b3-085b-4aac-a8f7-ade21f02e352>

In [51]:
distance_matrix_Jaccard

<artifact: DistanceMatrix uuid: e4865ce8-fe91-4ddf-95bc-1437d8c6c154>

In [55]:
# Perform stepwise RDA-ANOVA
es_all = {}
use_ = ['ATTRIBUTE_storagebuffer', 'ATTRIBUTE_host_subject_id']

# Clean up meta (only stuff to run)
mf_ord = md_round1and2_bothPS_df_shotgun_q2.to_dataframe().copy()
#mf_ord = mf_ord[mf_ord.index.isin(pcoa_res_shotgun['RPCA'].index)]
#mf_ord = mf_ord[mf_ord.index.isin(pcoa_res_shotgun['Jaccard'].index)]

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]
print(len(keep_))
# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_shotgun.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_) #, mf_ord_.columns)

# Concat output from all runs and export
es_alldf = pd.concat(es_all) #.rename({'+ sample_type2':'+ sample_type'}, axis=0)
# es_alldf.to_csv('/Users/Justin/Mycelium/UCSD/00_Knight_Lab/03_Extraction_test_12201/round_02/results/stepwise_anova/stepwise_anova_shotgun.txt', sep='\t')
es_alldf



3
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: /Users/caitrionabrennan/Downloads/assets/stepwise-rda.R /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpe8qg1taw/ord_.tsv /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpe8qg1taw/mf_.txt /var/folders/lh/d1kdwkz53856l7_mtjyz0f640000gn/T/tmpe8qg1taw/output.effect.size.tsv

R version 4.0.5 (2021-03-31) 


Loading required package: permute
Loading required package: lattice
This is vegan 2.5-7


Call: rda(formula = Y_16S ~ 1, data = X_16S, scale = TRUE)

              Inertia Rank
Total               3     
Unconstrained       3    3
Inertia is correlations 

Eigenvalues for unconstrained axes:
   PC1    PC2    PC3 
1.9164 0.7263 0.3573 

Call: rda(formula = Y_16S ~ ATTRIBUTE_storagebuffer +
ATTRIBUTE_host_subject_id, data = X_16S, scale = TRUE)

              Inertia Proportion Rank
Total          3.0000     1.0000     
Constrained    0.3602     0.1201    3
Unconstrained  2.6398     0.8799    3
Inertia is correlations 

Eigenvalues for constrained axes:
   RDA1    RDA2    RDA3 
0.27652 0.06878 0.01489 

Eigenvalues for unconstrained axes:
   PC1    PC2    PC3 
1.7569 0.5776 0.3053 



EmptyDataError: No columns to parse from file

In [26]:
pcoa_res_shotgun["RPCA"].shape,pcoa_res_shotgun["Jaccard"].shape

((84, 84), (84, 84))

In [27]:
mf_ord.columns

Index(['ATTRIBUTE_storagebuffer', 'ATTRIBUTE_host_subject_id',
       'ATTRIBUTE_sample_type2'],
      dtype='object')

In [28]:
s1=set(pcoa_res_shotgun['RPCA'].index)

In [29]:
s2=set(pcoa_res_shotgun['Jaccard'].index)

In [30]:
s1

{'361162770',
 '361162771',
 '361162790',
 '361162793',
 '361162798',
 '361162807',
 '361162809',
 '361164071',
 '361164072',
 '361164081',
 '361164082',
 '361164083',
 '361164086',
 '361164090',
 '361164091',
 '361164093',
 '361164094',
 '361164109',
 '361164117',
 '361164176',
 '361164180',
 '361164195',
 '361164201',
 '361164203',
 '361164213',
 '361164223',
 '361164224',
 '361164229',
 '361164231',
 '361164234',
 '361164244',
 '361164250',
 '361164254',
 '361164255',
 '361164259',
 '361164260',
 '363193206',
 '363193214',
 '363193215',
 '363193225',
 '363193232',
 '363193235',
 '363193242',
 '363193260',
 '363193261',
 '363193268',
 '363193269',
 '363193276',
 '363193575',
 '363193577',
 '363193584',
 '363193585',
 '363193593',
 '363193594',
 '363193595',
 '363193596',
 '363193602',
 '363193603',
 '363193604',
 '363193613',
 '363197846',
 '363197847',
 '363197866',
 '363197878',
 '363197879',
 '363197880',
 '363197881',
 '363197883',
 '363197885',
 '363197888',
 '363197889',
 '3631